In [1]:
import os
import math
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from itf import iTransformer
from itf_utils import Dataset_hour, train
from argparse import Namespace

# 1. loading data 
- long term forcast
- data: [ETT-small](https://github.com/zhouhaoyi/ETDataset/tree/main)

In [2]:
data_dir = "/home/scc/Downloads/Datas/ETDataset/ETT-small"
data_file = f'{data_dir}{os.sep}ETTh2.csv'
data_file = f'{data_dir}{os.sep}ETTh1.csv'
df = pd.read_csv(data_file)
df = df.sort_values(by='date', ignore_index=True)
print(df.shape)
df.head()

(17420, 8)


,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,2016-07-01 00:00:00,5.827,2.009,1.599,0.462,4.203,1.340,30.531000
1,2016-07-01 01:00:00,5.693,2.076,1.492,0.426,4.142,1.371,27.787001
2,2016-07-01 02:00:00,5.157,1.741,1.279,0.355,3.777,1.218,27.787001
3,2016-07-01 03:00:00,5.090,1.942,1.279,0.391,3.807,1.279,25.044001
4,2016-07-01 04:00:00,5.358,1.942,1.492,0.462,3.868,1.279,21.948000


In [3]:
base_ = 30 * 24
seq_len = 96
border1s = [0, 12 * base_ - seq_len, (12 + 4) * base_ - seq_len]
border2s = [12 * base_, (12 + 4) * base_, (12 + 8) * base_]
list(zip(border1s, border2s))

[(0, 8640), (8544, 11520), (11424, 14400)]

# dataloader

In [4]:
cfg = Namespace(
    task_name='long_term_forecast',
    output_attention=False,
    # enc param: embed & attention
    use_norm=True,
    d_model=512,
    d_ff=512,
    n_heads=16, #8,
    dropout=0.15,
    e_layers=2,
    activation='gelu',
    # seq param
    seq_len=4*7*24,
    pred_len=24, 
    learning_rate=5e-4, #5e-6,
    num_epochs=20,
    batch_size=32,
    device='cuda' if torch.cuda.is_available() else 'cpu',
    save_dir='./models_pt'
)
target_cols = ['HUFL', 'HULL', 'MUFL', 'MULL', 'LUFL', 'LULL', 'OT']

In [5]:
# df.columns

In [6]:
tr_data_set = Dataset_hour(df, target_cols=target_cols, flag='train', seq_len=cfg.seq_len, pred_len=cfg.pred_len, scale=True)
tr_loader = DataLoader(tr_data_set, batch_size=cfg.batch_size, shuffle=True)
print(len(tr_data_set))

val_data_set = Dataset_hour(df, target_cols=target_cols, flag='val', seq_len=cfg.seq_len, pred_len=cfg.pred_len, scale=True)
val_loader = DataLoader(val_data_set, batch_size=cfg.batch_size, shuffle=True)
print(len(val_data_set))

te_data_set = Dataset_hour(df, target_cols=target_cols, flag='test', seq_len=cfg.seq_len, pred_len=cfg.pred_len, scale=True)
test_loader = DataLoader(te_data_set, batch_size=cfg.batch_size, shuffle=False)
print(len(te_data_set))

set_type(0): 0 -> 8640 len=7945
df_raw.shape=(17420, 8) data.shape=(17420, 7) self.data_x.shape=(8640, 7)
7945
set_type(1): 7968 -> 11520 len=2857
df_raw.shape=(17420, 8) data.shape=(17420, 7) self.data_x.shape=(3552, 7)
2857
set_type(2): 10848 -> 14400 len=2857
df_raw.shape=(17420, 8) data.shape=(17420, 7) self.data_x.shape=(3552, 7)
2857


# train

In [7]:
itf_model = iTransformer(cfg)

In [8]:
itf_model = train(
    itf_model, tr_loader, val_loader, test_loader, cfg, invert_tf=te_data_set.inverse_transform
)

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 001, Steps: 249 | Train Loss: 0.28610(invLoss: 4.50089) Vali Loss: 0.51925 Test Loss: 0.36870
Saved Best Model: loss=0.51925
Epoch: 002, Steps: 249 | Train Loss: 0.19597(invLoss: 3.12794) Vali Loss: 0.56300 Test Loss: 0.38848
Epoch: 003, Steps: 249 | Train Loss: 0.14228(invLoss: 2.21126) Vali Loss: 0.51322 Test Loss: 0.37568
Saved Best Model: loss=0.51322
Epoch: 004, Steps: 249 | Train Loss: 0.11532(invLoss: 1.76922) Vali Loss: 0.53321 Test Loss: 0.37308
Epoch: 005, Steps: 249 | Train Loss: 0.10251(invLoss: 1.55412) Vali Loss: 0.51463 Test Loss: 0.36755
Epoch: 006, Steps: 249 | Train Loss: 0.09246(invLoss: 1.39085) Vali Loss: 0.49458 Test Loss: 0.36746
Saved Best Model: loss=0.49458
Epoch: 007, Steps: 249 | Train Loss: 0.08611(invLoss: 1.28549) Vali Loss: 0.51202 Test Loss: 0.36273
Epoch: 008, Steps: 249 | Train Loss: 0.08155(invLoss: 1.21609) Vali Loss: 0.48616 Test Loss: 0.36218
Saved Best Model: loss=0.48616
Epoch: 009, Steps: 249 | Train Loss: 0.07663(invLoss: 1.13138) Vali 

# inference 

In [15]:
print(len(test_loader))
i_set = 12 #24 #48
device = cfg.device
itf_model.eval()
with torch.no_grad():
    for i, (batch_x, batch_y, seq_x_mark, seq_y_mark) in enumerate(test_loader):
        batch_x = batch_x.float().to(device)
        batch_y = batch_y.float().to(device)
        seq_x_mark = seq_x_mark.float().to(device)
        seq_y_mark = seq_y_mark.float().to(device)

        outputs = itf_model(batch_x, seq_x_mark)
        pred = outputs.detach().cpu()
        true = batch_y.detach().cpu()
        if i == i_set:
            print(i)
            break

90
12


In [16]:
# df[:48].set_index('date')['OT'].plot()

In [17]:
import matplotlib.pyplot as plt


for b in range(pred.shape[0]):
    plt.figure(figsize=(16, 3))
    back_seq = tr_data_set.inverse_transform(batch_x[b].detach().cpu().numpy())[:, -1].flatten()
    pred_ = np.concatenate([np.array([np.nan]*len(back_seq)), tr_data_set.inverse_transform(pred[b].numpy())[:, -1].flatten()])
    real_only = np.concatenate([np.array([np.nan]*len(back_seq)), tr_data_set.inverse_transform(true[b].numpy())[-cfg.pred_len:, -1].flatten()])
    real_ = np.concatenate([back_seq, tr_data_set.inverse_transform(true[b].numpy())[-cfg.pred_len:, -1].flatten()])

    plt.plot(pred_, label='pred', linestyle='--')
    plt.plot(real_, label='real', linestyle='--', color='darkred', alpha=0.5)
    plt.plot(real_only, label='real', color='darkred', alpha=0.8)
    plt.legend()
    plt.title(f'sample i={i_set}')
    plt.show()
    break
    # if b >= 3:
    #     break